In [ ]:
# import moduel and data 
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
# 自用模組


def get_num_data(df):
    return df.select_dtypes("number")

def get_nonNum_data(df):
    return df.select_dtypes(exclude="number")

#                                  ==== category data 相關    ====  

#放入"全部"資料，與預測標的名稱  
# 回傳 None ,但於stdout上顯示各項指標
def catData_show_statistic(df,y_name):
    
    for i in df.select_dtypes(exclude="number"):
        miss = df.isna().mean(axis=0).to_frame("miss_rate").sort_values("miss_rate",ascending=False).transpose()
        print("==========================")
        print("欄位:",i)
        
        print("資料缺失率:",miss[i]["miss_rate"])
        
        
        counts = df[i].value_counts()
        print("共",len(counts),"種資料\n") 
        for c in counts.index:
            print(c, "數量:",counts[c])
            print("此類型的y平均數", df[y_name][df[i] == c].to_frame("1").mean()["1"] )
            print("此類型的y中位數", df[y_name][df[i] == c].to_frame("1").median()["1"] )
            
#放入"全部"資料，與預測標的名稱  
# 回傳 將類別資料改成其類別對y的中位數 (資料缺失填入0)  與 被更改的類別label-> value 的dir
#                                                      供接下來testing使用replace(dir)
def catData_encoding(df,y_name):
    replace_dir = {}
    for i in df.select_dtypes(exclude="number"):
        counts = df[i].value_counts()
        
        
        
        replace_dir[i] = {}
        
        for c in counts.index:
            replace_dir[i][c] = df[y_name][df[i] == c].to_frame("1").mean()["1"]

    return df.replace(replace_dir),replace_dir
        
        
        
        
        
#                                  ====   missing rate 相關   ====                    #

def show_miss_rate(df):
    miss = df.isna().mean(axis=0).to_frame("miss_rate")
    miss = miss[miss.miss_rate!=0].sort_values("miss_rate",ascending=False)
    sns.catplot(kind="bar",data=miss.transpose(),aspect=2,height=10)
    print(miss)
# 印出所有含有na或null的標籤(特徵)，並依缺失率高低排序 
# 同時畫出表格

def drop_miss_rate(df,threshold):
    miss = df.isna().mean(axis=0).to_frame("miss_rate")
    miss = miss[miss.miss_rate!=0].sort_values("miss_rate",ascending=False).transpose()
    
    miss_index = []
    for i in miss:
        if (miss[i]>threshold).bool():
            miss_index.append(i)
    return df.drop(miss_index , axis=1)
# 回傳丟掉高於threshold遺失率的欄位  
# 0 <= threshold <= 1



#   ===================================   for num data   ===================================  # 
#                                  ====   分布相關   ====                    #

    

def numData_show_statistic(df,y_name):
    print("比較標的:常態分佈 偏度與峰度皆為0")
    print("偏度很大=>極度右偏 ， 峰度很大 => 值集中")
    miss = df.isna().mean(axis=0).to_frame("miss_rate").sort_values("miss_rate",ascending=False).transpose()

    for i in df:
        print("==========================")
        print("欄位:",i)
        print("    ---     ")
        print("平均值",df[i].mean())
        print("標準差",df[i].std())
        print("偏度:",df[i].skew())
        print("峰度:",df[i].skew())
        print("    ---     ")
        print("資料缺失率:",miss[i]["miss_rate"])
        print("    ---     ")
        print("Pearson相關係數(共線程度):",df.corr(method='pearson')[y_name][i])
        print("Spearman相關係數(廣義關係):",df.corr(method='spearman')[y_name][i])
        print("Kendall相關係數(單調關係):",df.corr(method='kendall')[y_name][i])
# 照欄位順序依次印出基本統計數據

#如名，印出y_mean 與 y_std 關係，確認是否要取log  
#(如果無關=>不用調  正比就開根、正二次方就取log)
def show_y_mean_and_std_relation(df,y_name):
    temp = {}
    temp["std"] = []
    temp["mu"] = []
    
    print("Y偏度:",df[y_name].skew())
    print("Y峰度:",df[y_name].kurt())

    #pd.set_option('display.float_format', lambda x: '%.3f' % x)

    for i in range(len(df[y_name])):

        temp["mu"].append(df[y_name][:i+1].mean())
        temp["std"].append(df[y_name][:i+1].std()*train_y[:i+1].std()  )

    temp = pd.DataFrame(temp) 
    print(temp)

    #from matplotlib import pyplot as plt
    #f, ax = plt.subplots()
    #ax.ticklabel_format(style='plain', axis='both')
    sns.relplot(x="std", y="mu", data=temp)


def numData_show_detail(df,x_name,y_name):
    des = df.describe()
    
    print("===============與目標關係程度===============\n\n")
    print("Pearson相關係數(共線程度):",df.corr(method='pearson')[y_name][x_name])
    print("Spearman相關係數(單調關係):",df.corr(method='spearman')[y_name][x_name])
    print("Kendall相關係數(單調關係):",df.corr(method='kendall')[y_name][x_name])
    print("\n")
    print("================基本數值================\n\n")
    print(des[x_name])
    print("\n")
    print("X偏度:",df[x_name].skew())
    print("X峰度:",df[x_name].skew())
    print("================與目標關係圖================\n\n")
    sns.relplot(x=x_name, y=y_name, data=df)
    print("\n")
# 單欄位詳細資料

# 輸入想要的correlation係數絕對值門檻，指定方法
# 回傳高於門檻的 "index label"(list)  
def numData_corr_filter(df,y_name,threshold,kind="spearman"):
    train_x_label = []
    for i in df:
 
        if abs(df.corr(method=kind)[y_name][i]) > threshold :
             if i != y_name:
                train_x_label.append(i)
    return train_x_label


# 限制所有值在n個標準差之內
# 回傳限制完的結果(df)
def numData_outlier_limit(df ,y_name , n):
    
    #label_counter = 0
    #total_labels = len(df.columns)
    
    for i in df:
        
        
        if i!=y_name:
            mu = df[i].mean()
            std = df[i].std()
            for j in range(len(df[i])):
                if df[i][j]<mu :
                    df.loc[j,i] = max(df[i][j],mu-n*std)


                if df[i][j]>mu:
                    df.loc[j,i] = min(df[i][j],mu+n*std)
        #label_counter+=1
        #print("Dealing outlier (Lable):",label_counter,"\\",total_labels)
    return df

# ===================================   for data processing   =================================== 
def z_score(df):
    return (df-df.mean())/df.std()
# 回傳標準化z-score後的表格(df)

def log(df):
    logdf = np.log(df+1)
    logdf= logdf.replace([np.inf, -np.inf], np.nan)
    
    return logdf.fillna(0)
# 回傳log transform 結果，0保持在0

def shuffle_data(df): 
    return df.sample(frac=1).reset_index(drop=True)
# 隨機排序data

def split_data(df,len_train_data):
    len_all_data = len(df.index)
    
    s = pd.Series( [i for i in range(0,len_all_data-len_train_data)])
    train = all_data.loc[:len_train_data]

    test = all_data.loc[len_train_data:].set_index(s)
    
    return train,test
# 將data 切成train與test,並且給定長度



# plot the raw observations
def cross_validation_plot(err_record):
    """
    input : err_record : { size1 : [err1 , err2] , size2: [err1 , err2] ... }
    output : print a plot  [Cross-validation error VS parameter size]
    """
    parameter_size = list(err_record.keys())
    for size in parameter_size:
        errs = err_record[size]
        plt.scatter([ size] * len(errs), errs)

    # plot the trend line with error bars that correspond to standard deviation
    err_mean = np.array([np.mean(v) for k,v in sorted(err_record.items())])
    err_std = np.array([np.std(v) for k,v in sorted(err_record.items())])
    plt.errorbar(parameter_size, err_mean ,  yerr=err_std)
    plt.title('Cross-validation on parameter size')
    plt.xlabel('parameter size')
    plt.ylabel('Cross-validation error')
    plt.show()

In [ ]:
# get data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")





train_y = train["SalePrice"]


"""
# data preprocessing  for  
1.encoding cat data by median
2.nan remove (by drop and replace)
3.scaling (take z_score)
4.outlier  
"""
# encoding
(train_x,replace_dir) = catData_encoding(train,"SalePrice")
# drop too many nan column 
train_x = drop_miss_rate(train_x , 0.2)
# nan remove
train_x = train_x.fillna(0)
# outlier 
train_x = numData_outlier_limit(train_x,"SalePrice",3)
print(train_x)
# scaling 
train_x = z_score(train_x)

# correlation
train_x_label = numData_corr_filter(train_x ,"SalePrice",0.3)



#train_x


train_x = train_x[train_x_label]


#temp["SalePrice"] = train_y
#numData_show_detail(temp,"3SsnPorch","SalePrice")
#print(train_x_corr["3SsnPorch"])

In [ ]:
# training part  -- with cross fold validation

train_y_log = np.log(train_y.to_numpy())

num_fold = 5
print(train_x.shape)
train_x_fold = np.array_split(train_x.to_numpy(),num_fold)
train_y_fold = np.array_split(train_y_log,num_fold)


test_h_size  = [i for i in range(2,10)]
import sklearn

#from sklearn.linear_model import Ridge
#reg = Ridge(alpha=0.2)
#from sklearn import linear_model
#reg = linear_model.LinearRegression()
#from sklearn.linear_model import LogisticRegression
#reg = LogisticRegression(max_iter=2000)
#from sklearn import linear_model
#reg = linear_model.ARDRegression()
#from sklearn.ensemble import RandomForestRegressor
#reg = RandomForestRegressor(max_depth=4, random_state=0)
#from sklearn.ensemble import GradientBoostingRegressor
#reg = GradientBoostingRegressor(n_estimators=2048, learning_rate=0.01,\
#max_depth=2, random_state=0, loss='ls')

err_record = {} # 共有 #待測參數 個 list ,每個list含 #fold 筆error 資料

from sklearn.neural_network import MLPRegressor

for h_size in test_h_size:
                                                                  # target 
    reg = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=  h_size  ,max_iter=4000,alpha=1,verbose=False ,learning_rate="adaptive")
    
#from sklearn import svm
#reg=svm.SVR()
#from sklearn import tree
#reg = tree.DecisionTreeRegressor()
#reg = sklearn.neural_network.MLPRegressor(tol=1e-7,learning_rate_init=0.06428,hidden_layer_sizes=(60,30,15,7,3,1),max_iter=15000,verbose=True)
#print(train_x)
#reg.fit(train_x, np.sqrt(train_y))
    err_record[h_size]=[]
    for n in range(num_fold):
          
     
        train_x_split = np.concatenate(train_x_fold[:n]+train_x_fold[n+1:])
        train_y_split = np.concatenate(train_y_fold[:n]+train_y_fold[n+1:])
        reg.fit(train_x_split, train_y_split)
        
        val_hat = reg.predict(train_x_fold[n])
        test_err = mean_squared_log_error(train_y_fold[n], val_hat)
        err_record[h_size].append(test_err)
        
    print(err_record)

In [ ]:
cross_validation_plot(err_record)

In [ ]:
# choose best hyperparameter
best_size = test_h_size[err_mean.argmin()] # because less error is better
print(best_size)

In [ ]:
reg = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=512 ,max_iter=4000,alpha=0.6,verbose=False ,learning_rate="adaptive")
reg.fit(train_x , train_y_log)

In [ ]:
test = pd.read_csv("test.csv")


temp = {"Id":[],"SalePrice":[]}

test_x = test
# encoding cat data
test_x = test_x.replace(replace_dir)
# nan remove
test_x = test_x.fillna(0)


# get relative feature
test_x = test_x[train_x_label]
# oulier dealing
test_x = numData_outlier_limit(test_x,"",3)
test_x  = z_score(test_x)
#print(test_x)




#print(num_data_test)

#test_x  = log(num_data_test).replace([np.inf, -np.inf], np.nan).fillna(0)


test_y_hat = reg.predict(test_x)

count = 0
for i in test_y_hat:
    
    temp["Id"].append(test["Id"][count])
    
    #temp["SalePrice"].append(i**2)
    temp["SalePrice"].append(np.exp(i))
    count+=1

    
#print(temp)
#i = np.random.randint(0,len(temp["Id"]))
i=0
print("第",i,"筆",temp["Id"][i],temp["SalePrice"][i])
temp = pd.DataFrame(temp)

#for i in range(len(temp["Id"])):
#    print("第一筆",temp["Id"][i],temp["SalePrice"][i])



#test_y 
#mean_squared_error()

In [ ]:
train_y_hat = reg.predict(train_x)

for i in range(len(train_y_hat)):
    #train_y_hat[i]=(train_y_hat[i])**2
    train_y_hat[i]=np.exp(train_y_hat[i])
for i in range(len(test_y_hat)):
    #test_y_hat[i]=(test_y_hat[i])**2
    test_y_hat[i]=np.exp(test_y_hat[i])    

# validation variance error (overfitting) or bias error (underfiting)
from sklearn.metrics import mean_squared_log_error
train_err = mean_squared_log_error(train_y , train_y_hat)





test_y = pd.read_csv("solution.csv")



test_err = mean_squared_log_error(test_y["SalePrice"] , test_y_hat)

print("training error:",train_err)
print("testing error :",test_err)

In [ ]:
!del answer.csv
temp.to_csv("answer.csv",index=False)

In [ ]:
#saving and loading part 

import joblib
joblib.dump(reg,'svm_0.41.joblib')
#reg2 =joblib.load("NNmodel_loss_0.18.pkl")
#reg2.predict(num_data_test)[0]

In [ ]:
#ensemble
import sklearn
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
#reg = sklearn.neural_network.MLPRegressor(max_iter=15000)
from sklearn import svm
from sklearn.ensemble import VotingRegressor
from sklearn import tree

from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(max_depth=4, random_state=0)

from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=75, learning_rate=0.01,\
max_depth=1, random_state=0, loss='ls')

#tree = tree.DecisionTreeRegressor()
lr = linear_model.LinearRegression()
lr_ridge  = Ridge(alpha=0.2)
lr_bay  = linear_model.ARDRegression() 
#svm = svm.SVR()
nn = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=16,max_iter=4000,alpha=1,verbose=True ,learning_rate="adaptive")
ereg = VotingRegressor(estimators=[('nn',nn),('lr_ridge',lr_ridge),('lr_bay',lr_bay),("forest",forest) ,('lr',lr),('gbr',gbr)])
ereg = ereg.fit(train_x, np.log(train_y))

test_y_hat = ereg.predict(test_x)

temp = {"Id":[],"SalePrice":[]}
count = 0

for i in test_y_hat:
    
    temp["Id"].append(test["Id"][count])
    
    #temp["SalePrice"].append((i)**2)
    temp["SalePrice"].append(np.exp(i))
    count+=1

    
#print(temp)
print("第一筆",temp["Id"][0],temp["SalePrice"][0])
temp = pd.DataFrame(temp)
!del answer.csv
temp.to_csv("answer.csv",index=False)

train_y_hat = ereg.predict(train_x)

for i in range(len(train_y_hat)):
    #train_y_hat[i]=(train_y_hat[i])**2
    train_y_hat[i]=np.exp(train_y_hat[i])
for i in range(len(test_y_hat)):
    #test_y_hat[i]=(test_y_hat[i])**2
    test_y_hat[i]=np.exp(test_y_hat[i])    

# validation variance error (overfitting) or bias error (underfiting)
from sklearn.metrics import mean_squared_log_error
train_err = mean_squared_log_error(train_y , train_y_hat)


    


test_y = pd.read_csv("solution.csv")



test_err = mean_squared_log_error(test_y["SalePrice"] , test_y_hat)

print("training error:",train_err)
print("testing error :",test_err)

for e in ereg.transform(test_x.iloc[:1,:]):
    print(np.exp(e))

In [ ]:
import sklearn


print('The scikit-learn version is {}.'.format(sklearn.__version__))